# High-Level Summary of Foul Analysis

This notebook performs a comprehensive spatial and statistical analysis of foul events in major European football leagues during the 2015/2016 season, excluding Champions League matches.

The analysis includes:

- **League-wide Foul Count Comparison:** Using Poisson regression and Holm adjustment to compare the average number of fouls per match across different leagues.
- **Horizontal Pitch Symmetry Test:** Employing GLMs and Holm adjustment to test for differences in foul counts between mirrored halves of the pitch within each league.
- **Poisson Point Process & Spatial Intensity:** Utilizing Kernel Density Estimation (KDE) to create intensity heatmaps visualizing the spatial distribution of fouls for each league.
- **Bandwidth Selection for KDE:** Investigating the effect of different bandwidths on the intensity maps and assessing model performance using techniques like GridSearchCV and Quadrat tests.
- **Simulating Match Foul Patterns:** Using the thinning algorithm to simulate typical foul distributions in a single game for each league based on estimated intensity.
- **Inter-League Spatial Comparison:** Comparing the spatial distribution of fouls between leagues using intensity ratio surfaces.
- **Data-Driven Zoning:** Applying Gaussian Mixture Models (GMM) to identify data-driven zones on the pitch based on foul intensity.
- **Zonal Statistical Testing:** Conducting permutation tests with Energy Distance to statistically compare the spatial distribution of fouls between leagues within predefined zones, with p-values adjusted using the Holm method.
- **Overall Comparison using Stouffer's Z:** Applying Stouffer's Z method to combine p-values across zones for a combined measure of spatial differences between leagues.

Overall, the notebook provides a detailed exploration of where and how frequently fouls occur in different leagues, employing various statistical and spatial analysis techniques.

 **Package Imports**

In [ ]:
!pip install statsbombpy
!pip install mplsoccer
!pip install highlight_text

In [ ]:
import pandas as pd
import statsbombpy as sb
from mplsoccer import Pitch
from highlight_text import ax_text, fig_text
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.multitest import multipletests
from itertools import combinations
import numpy as np
from scipy.stats import norm
from sklearn.neighbors import KernelDensity
from sklearn.mixture import GaussianMixture
from scipy.spatial.distance import cdist
import matplotlib.patches as patches
from scipy.interpolate import RegularGridInterpolator
from sklearn.model_selection import train_test_split
from scipy.stats import chi2
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
from patsy import dmatrix
from sklearn.model_selection import GridSearchCV
from matplotlib.cm import ScalarMappable
import matplotlib.patches as patches

**import data**

This section of the notebook imports the necessary data for the analysis. The data is stored in multiple CSV files in Google Drive, each potentially containing event data for different matches or seasons. The files are loaded into pandas DataFrames and then concatenated into a single DataFrame (df_concatenated2) for further processing.

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# Specify the path where the CSV file is located in your Google Drive
file_path1 = f'/content/drive/My Drive/referee_analysis1.csv'
file_path2 = f'/content/drive/My Drive/referee_analysis2.csv'
file_path3 = f'/content/drive/My Drive/referee_analysis3.csv'
file_path4 = f'/content/drive/My Drive/referee_analysis4.csv'
file_path5 = f'/content/drive/My Drive/referee_analysis5.csv'
file_path6 = f'/content/drive/My Drive/referee_analysis6.csv'
file_path7 = f'/content/drive/My Drive/referee_analysis7.csv'
file_path8 = f'/content/drive/My Drive/referee_analysis8.csv'
file_path9 = f'/content/drive/My Drive/referee_analysis9.csv'
file_path10 = f'/content/drive/My Drive/referee_analysis10.csv'
file_path11 = f'/content/drive/My Drive/referee_analysis11.csv'

# Read the CSV file into a DataFrame
df_imported1 = pd.read_csv(file_path1)
df_imported2 = pd.read_csv(file_path2)
df_imported3 = pd.read_csv(file_path3)
df_imported4 = pd.read_csv(file_path4)
df_imported5 = pd.read_csv(file_path5)
df_imported6= pd.read_csv(file_path6)
df_imported7 = pd.read_csv(file_path7)
df_imported8 = pd.read_csv(file_path8)
df_imported9 = pd.read_csv(file_path9)
df_imported10 = pd.read_csv(file_path10)
df_imported11 = pd.read_csv(file_path11)


In [ ]:

dfs_to_concat = [
    df_imported1,
    df_imported2,
    df_imported3,
    df_imported4,
    df_imported5,
    df_imported6,
    df_imported7,
    df_imported8,
    df_imported9,
    df_imported10,
    df_imported11
]

df_concatenated2 = pd.concat(dfs_to_concat, ignore_index=True)


**data selection and engineering**

This section focuses on selecting the relevant data for the analysis and engineering new features. We filter the concatenated dataset to include only the '2015/2016' season and exclude matches from the 'Europe - Champions League' competition to focus on major domestic leagues. A binary flag `is_foul_committed` is created to easily identify foul events. Additionally, we transform the geospatial coordinates (`x`, `y`) to `x2` and `y2` by pivoting the pitch by 180 degrees. This transformation is useful for analyzing events from a consistent attacking direction, aiding in visualizations and spatial analysis where directionality is important.;



ensure selection of correct data

In [ ]:
df_concatenated2_2015_2016 = df_concatenated2[df_concatenated2['season'] == '2015/2016'].copy()

create flag for foul data

In [ ]:
df_concatenated2_2015_2016['is_foul_committed'] = (df_concatenated2_2015_2016['field'] == 'Foul_Committed').astype(int)

exclude champions league data

In [ ]:
df_concatenated2_2015_2016 = df_concatenated2_2015_2016[
    (df_concatenated2_2015_2016['competition'] != 'Europe - Champions League')
].copy()


pivot data 180 degrees for visualisation purposes

In [ ]:
df_concatenated2_2015_2016['x2']=abs(df_concatenated2_2015_2016['x']-120)
df_concatenated2_2015_2016['y2']=abs(df_concatenated2_2015_2016['y']-80)

**League foul count comparison by Poisson Regression and Holm Adjustment**

This section compares the average number of fouls committed per match across different major European leagues using Poisson regression. Poisson regression is a generalized linear model (GLM) suitable for modeling count data, like the number of fouls in a match, which are non-negative integers. The model estimates the relationship between the league (as a categorical variable) and the log of the expected foul count.

The coefficients from the Poisson model represent the change in the log of the expected foul count relative to the reference category (in this case, 'England - Premier League' as it is the first alphabetically). Exponentiating these coefficients gives the rate ratio, which can be interpreted as how many times more or less likely a foul is to occur in a given league compared to the reference league.

We also examine the residual deviance and degrees of freedom to check for overdispersion. A ratio significantly greater than 1 suggests that the variance of the foul counts is higher than expected under the Poisson distribution, indicating potential overdispersion. While the Poisson model provides a useful baseline, overdispersion might suggest that a Negative Binomial model could be a better fit.

Finally, we perform pairwise comparisons between the leagues and adjust the p-values using the Holm-Bonferroni method. The Holm adjustment is a multiple comparison procedure that controls the family-wise error rate (FWER), reducing the chance of making a Type I error (false positive) when performing multiple hypothesis tests simultaneously. The adjusted p-values provide a more conservative measure of statistical significance for each comparison.

Get data for poisson regression test

In [ ]:
foul_committed_per_match = df_concatenated2_2015_2016[df_concatenated2_2015_2016['field'] == 'Foul_Committed'].groupby(['match_id', 'competition']).agg(
    total_fouls=('is_foul_committed', 'sum')
).reset_index()

print(foul_committed_per_match.head())


summarise data for context

In [ ]:
foul_committed_avg_by_competition = foul_committed_per_match.groupby('competition')['total_fouls'].mean().reset_index()
foul_committed_avg_by_competition = foul_committed_avg_by_competition.rename(columns={'total_fouls': 'average_fouls'})

print("Average Fouls Committed by Competition (2015/2016 Season):")
foul_committed_avg_by_competition

run poisson regression

In [ ]:

# Fit a Poisson regression model
poisson_model = smf.glm(formula='total_fouls ~ C(competition)', data=foul_committed_per_match, family=sm.families.Poisson()).fit()


# Summary with adjusted standard errors
print(poisson_model.summary())

In [ ]:

# To check for overdispersion, you can compare the residual deviance to the degrees of freedom.
# If the residual deviance is significantly larger than the degrees of freedom, it suggests overdispersion.

# Get the residual deviance and degrees of freedom from the Poisson model summary
residual_deviance = poisson_model.deviance
degrees_of_freedom = poisson_model.df_resid

print(f"\nResidual Deviance: {residual_deviance}")
print(f"Degrees of Freedom: {degrees_of_freedom}")

# A common rule of thumb is to check the ratio of residual deviance to degrees of freedom.
# If this ratio is significantly greater than 1 (e.g., > 1.5 or 2, depending on context),
# overdispersion might be present.
deviance_ratio = residual_deviance / degrees_of_freedom
print(f"Deviance Ratio (Residual Deviance / Degrees of Freedom): {deviance_ratio:.4f}")

# You can also perform a formal statistical test for overdispersion,
# such as the dispersion test from statsmodels.
# Note: The dispersion test is often more appropriate for count data models.
# If you suspect overdispersion, a Negative Binomial model is a common alternative.

# Let's fit a Negative Binomial model to compare
try:
    negbin_model = smf.glm(formula='total_fouls ~ C(competition)', data=foul_committed_per_match, family=sm.families.NegativeBinomial()).fit()
    print("\nNegative Binomial Model Summary:")
    print(negbin_model.summary())

    # Compare AIC values as a measure of model fit (lower is generally better)
    print(f"\nPoisson Model AIC: {poisson_model.aic:.4f}")
    print(f"Negative Binomial Model AIC: {negbin_model.aic:.4f}")

except Exception as e:
    print(f"Could not fit Negative Binomial model. Error: {e}")

# Interpretation:
# - If the Deviance Ratio is close to 1, the Poisson model might be appropriate.
# - If the Deviance Ratio is significantly greater than 1, the Negative Binomial model
#   is likely a better fit for the overdispersed count data.
# - Compare the AIC values: the model with the lower AIC generally provides a better fit.


In [ ]:
# Unique levels of the categorical variable
levels = foul_committed_per_match['competition'].unique()

# Create DataFrame to predict at each level
new_data = pd.DataFrame({'competition': levels})
new_data['predicted_mean'] = poisson_model.predict(new_data)
new_data['linear_predictor'] = poisson_model.predict(new_data, linear=True)

In [ ]:
# Get design matrix
X = dmatrix(poisson_model.model.data.design_info.builder, new_data)

# Variance-covariance matrix of model
cov = poisson_model.cov_params()

# Standard errors of linear predictions
se = np.sqrt(np.diag(np.dot(X, np.dot(cov, X.T))))
new_data['se'] = se

In [ ]:

results = []

for (i, j) in combinations(range(len(levels)), 2):
    comp1 = new_data.iloc[i]
    comp2 = new_data.iloc[j]

    diff = comp1['linear_predictor'] - comp2['linear_predictor']
    se_diff = np.sqrt(comp1['se']**2 + comp2['se']**2)
    z = diff / se_diff
    p = 2 * (1 - norm.cdf(np.abs(z)))  # <-- Use norm from scipy.stats

    results.append({
        'comparison': f"{comp1['competition']} - {comp2['competition']}",
        'mean_diff (log scale)': diff,
        'z': z,
        'raw_p': p
    })

In [ ]:

# Convert the list of results to a DataFrame
comp_df = pd.DataFrame(results)

# Adjust p-values using Holm correction (strong control of family-wise error rate)
comp_df['p_adj'] = multipletests(comp_df['raw_p'], method='holm')[1]

# Determine which comparisons are significant after adjustment
comp_df['significant'] = comp_df['p_adj'] < 0.05

# Optionally, get the ratio of means (back-transformed from log scale)
comp_df['rate_ratio'] = np.exp(comp_df['mean_diff (log scale)'])

In [ ]:
print(comp_df[['comparison',
               'mean_diff (log scale)',
               'rate_ratio',
               'z',
               'raw_p',
               'p_adj',
               'significant']])

In [ ]:
comp_df[['raw_p', 'p_adj']] = comp_df[['raw_p', 'p_adj']].round(3)


In [ ]:

# Select and print the desired columns
comp_df[['comparison', 'raw_p', 'p_adj', 'significant']]

**Horizontal pitch statisitical test by Poisson regression and holm adjustment**

This section investigates whether the spatial distribution of fouls is symmetrical across the horizontal axis of the pitch within each league. To do this, we divide the pitch into mirrored zones and compare the foul counts in these corresponding areas. We create variables like `y_shift` to measure distance from the center line, `zone` to define broad pitch areas (e.g., 'x1_y1', 'x2_y2'), `y_check` to distinguish between the two horizontal halves, and `competition_check` and `zone_pair` to group mirrored zones within each league. The `side` variable labels whether a zone is on the 'original' or 'mirrored' side of the pitch.

For each league (`zone_pair`), we fit a Generalized Linear Model (GLM) with a Poisson distribution to test the hypothesis that there is no difference in the expected number of fouls between the 'original' and 'mirrored' sides of the pitch (`total_fouls ~ C(side)`). The coefficient for `C(side)[T.mirrored]` in each model represents the estimated difference in the log of the expected foul count between the mirrored and original sides for that specific league's zone pair.

We then calculate p-values for each league's comparison. To account for multiple comparisons across the different league pairs, we apply the Holm adjustment method to the p-values. A significant adjusted p-value indicates a statistically significant difference in the spatial distribution of fouls between the original and mirrored sides for that league's zone pair.

In [ ]:
df_concatenated2_2015_2016['y_shift'] = abs(df_concatenated2_2015_2016['y'] - 0.5 * df_concatenated2_2015_2016['y'].max())
df_concatenated2_2015_2016['zone'] = ''

# Define the boundaries
x_boundaries = [0,  60, 120]
y_adj_boundaries = [0, 20, 40] # Using the minimum and maximum possible y_adj as boundaries

# Iterate through rows to assign zones
for index, row in df_concatenated2_2015_2016.iterrows():
    x = row['x']
    y_adj = row['y_shift']

    # Determine x zone
    x_zone = ''
    if x <= x_boundaries[1]:
        x_zone = 'x1'
    else:
        x_zone = 'x2'

    # Determine y_adj zone
    y_adj_zone = ''
    if y_adj <= y_adj_boundaries[1]:
        y_adj_zone = 'y1'
    else:
        y_adj_zone = 'y2'


    # Combine to create the zone
    df_concatenated2_2015_2016.at[index, 'zone'] = f'{x_zone}_{y_adj_zone}'

print(df_concatenated2_2015_2016[['x', 'y_shift', 'zone']].head())
print("\nValue counts for the new 'zone' field:")
print(df_concatenated2_2015_2016['zone'].value_counts())

In [ ]:
df_concatenated2_2015_2016['y_check'] = df_concatenated2_2015_2016['y'].apply(lambda y: 'a' if y < 40 else 'b')
foul_zone_per_match = df_concatenated2_2015_2016.groupby(['match_id', 'zone','y_check']).agg(
    total_fouls=('is_foul_committed', 'sum')
).reset_index()

print(foul_zone_per_match.head())

In [ ]:

df_concatenated2_2015_2016['competition_check'] = ''

# Define the zones and corresponding values
competition_mapping = {
    ('Spain - La Liga', 'a'): 'A',
    ('Spain - La Liga', 'b'): 'A*',
    ('England - Premier League', 'a'): 'B',
    ('England - Premier League', 'b'): 'B*',
    ('France - Ligue 1', 'a'): 'C',
    ('France - Ligue 1', 'b'): 'C*',
    ('Italy - Serie A', 'a'): 'D',
    ('Italy - Serie A', 'b'): 'D*',
    ('Germany - 1. Bundesliga', 'a'): 'E',
    ('Germany - 1. Bundesliga', 'b'): 'E*',
}

# Apply the mapping to create the 'zone_check' column
for index, row in df_concatenated2_2015_2016.iterrows():
    competition_key = (row['competition'], row['y_check'])
    if competition_key in competition_mapping:
        df_concatenated2_2015_2016.at[index, 'competition_check'] = competition_mapping[competition_key]


print(df_concatenated2_2015_2016[['competition', 'y_check', 'competition_check']].head())
print("\nValue counts for the new 'zone_check' field:")
print(df_concatenated2_2015_2016['competition_check'].value_counts())

In [ ]:
zone_pairs = {
    'A': 'A_pair',
    'A*': 'A_pair',
    'B': 'B_pair',
    'B*': 'B_pair',
    'C': 'C_pair',
    'C*': 'C_pair',
    'D': 'D_pair',
    'D*': 'D_pair',
    'E': 'E_pair',
    'E*': 'E_pair'
}
df_concatenated2_2015_2016['zone_pair'] = df_concatenated2_2015_2016['competition_check'].map(zone_pairs)
df_concatenated2_2015_2016['side'] = df_concatenated2_2015_2016['competition_check'].apply(lambda z: 'original' if not z.endswith('*') else 'mirrored')

In [ ]:
foul_zone_per_match = df_concatenated2_2015_2016.groupby(['match_id', 'zone_pair','side']).agg(
    total_fouls=('is_foul_committed', 'sum')
).reset_index()

print(foul_zone_per_match.head())

In [ ]:

results = []
foul_zone_per_match['side'] = pd.Categorical(foul_zone_per_match['side'], categories=['original', 'mirrored'])
for pair in foul_zone_per_match['zone_pair'].unique():
    subset = foul_zone_per_match[foul_zone_per_match['zone_pair'] == pair]

    if subset['side'].nunique() < 2:
        continue  # skip if no paired data

    model = smf.glm(
        formula='total_fouls ~ C(side)',
        data=subset,
        family=sm.families.Poisson()
    ).fit()

    coef = model.params['C(side)[T.mirrored]']
    se = model.bse['C(side)[T.mirrored]']
    z = coef / se
    p = 2 * (1 - norm.cdf(np.abs(z)))  # <-- Use norm from scipy.stats

    results.append({
        'zone_pair': pair,
        'coef_diff_log': coef,
        'z': z,
        'p': p
    })

# Convert to DataFrame
import pandas as pd
results_df = pd.DataFrame(results)

# Optionally apply FDR correction
from statsmodels.stats.multitest import multipletests
results_df['adjusted_p'] = multipletests(results_df['p'], method='holm')[1]

print(results_df)

In [ ]:
residual_deviance = model.deviance
degrees_of_freedom = model.df_resid
print(f"\nResidual Deviance: {residual_deviance}")
print(f"Degrees of Freedom: {degrees_of_freedom}")
deviance_ratio = residual_deviance / degrees_of_freedom
print(f"Deviance Ratio (Residual Deviance / Degrees of Freedom): {deviance_ratio:.4f}")

**Poisson Point Process**

This transforms our data using the geospatial x,y coordinates found in data into an intensity heatmap

A Poisson Point Process is a mathematical model used to describe the random distribution of points in space or time. In this context, we are using it to model the locations of fouls on the football pitch as points in a 2D spatial area. The intensity heatmap represents the estimated rate or density of fouls across the pitch, where higher intensity indicates areas where fouls are more likely to occur. Kernel Density Estimation (KDE) is a non-parametric technique used here to estimate this continuous intensity function from the discrete foul event locations.

**Investigating lambdas**
Finding the balance point between the best performing modelled lambda and one that makes visual sense

This section explores the effect of the bandwidth parameter (often denoted as $\lambda$ or $h$) in Kernel Density Estimation (KDE) on the resulting foul intensity maps. The bandwidth is a crucial parameter that controls the smoothness of the estimated density function. A smaller bandwidth results in a more localized and potentially noisy estimate that captures fine-grained variations, while a larger bandwidth produces a smoother, more generalized estimate that might obscure local patterns. The goal here is to investigate how different bandwidths influence the appearance of the foul intensity map and consider what bandwidth provides a good balance between capturing spatial patterns and visual interpretability, potentially guided by methods like cross-validation (as explored later).

**visualising different lambdas**

Here we test different lambdas on the Spanish - La Liga data

This visualization demonstrates the impact of varying the bandwidth parameter on the estimated foul intensity map for Spain - La Liga. By comparing the heatmaps generated with different bandwidths (e.g., 1, 2, 3, 5, 7.5, 10), we can observe how the level of detail and smoothness changes. A very small bandwidth (e.g., 1) shows individual foul clusters more distinctly but can appear noisy. As the bandwidth increases, the heatmap becomes smoother, revealing broader areas of high and low intensity. This visual exploration, along with quantitative measures (like those from GridSearchCV), helps in selecting an appropriate bandwidth for further analysis that best represents the underlying spatial distribution of fouls.

In [ ]:

df_fouls_spain = df_concatenated2_2015_2016[
    (df_concatenated2_2015_2016['is_foul_committed'] == 1) &
    (df_concatenated2_2015_2016['competition'] == 'Spain - La Liga')
].copy()

geospatial_features = ['x2', 'y2']

# Create a Pitch object for plotting
pitch = Pitch(pitch_type='statsbomb', pitch_color='grass', line_color='white', stripe=True)

# Define bandwidths for KDE
bandwidths = [1, 2, 3, 5, 7.5, 10]

# Create a figure with a 2x3 grid of subplots
fig, ax = plt.subplots(2, 3, figsize=(15, 6))

# Flatten the axes array for easier iteration
ax = ax.flatten()

# Loop through each bandwidth and plot on a new subplot
for i, h in enumerate(bandwidths):
    current_ax = ax[i]

    # Draw the pitch on the current subplot
    pitch.draw(ax=current_ax)

    # Get coordinates for KDE
    coords = df_fouls_spain[geospatial_features].values
    N_obs = len(coords)
    # Create a grid to evaluate the KDE on
    X_grid, Y_grid = np.meshgrid(np.linspace(0, 120, 240), np.linspace(0, 80, 160))
    grid_coords = np.vstack([X_grid.ravel(), Y_grid.ravel()]).T

    # Fit KDE model and get intensity
    kde = KernelDensity(kernel='gaussian', bandwidth=h)
    kde.fit(coords)
    log_dens = kde.score_samples(grid_coords)
    pdf = np.exp(log_dens).reshape(X_grid.shape)
            # === Convert density to intensity ===
    lambda_hat = N_obs * pdf
    # Plot the heatmap
    pcm = current_ax.imshow(lambda_hat, origin='lower', extent=(0, 120, 0, 80),
                            cmap='viridis', aspect='auto', alpha=0.8)

    # Add a colorbar for the current subplot
    fig.colorbar(pcm, ax=current_ax, label='Estimated Foul Intensity', shrink=0.7)

    # Add a title for the current subplot, including the bandwidth
    current_ax.set_title(f'Foul Intensity Map (BW={h})')

# Set a main title for the entire figure
fig.suptitle('Effect of Bandwidth on Foul Intensity Map for Spain - La Liga', fontsize=18)

# Adjust layout and display the plot
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

**performance deteration**

Looking at the overall model performance consideration different lambdas for all foul data

In [ ]:
# Filter fouls for 2015/2016 season
df_fouls = df_concatenated2_2015_2016[df_concatenated2_2015_2016['is_foul_committed'] == 1].copy()
geospatial_features = ['x2', 'y2']
unique_competitions = df_fouls['competition'].unique()

# Create Pitch object once
pitch = Pitch(pitch_color='grass', line_color='white', stripe=True)
bandwidths = np.linspace(0.1, 10.0, 30)
# Dictionary to store intensity data and best bandwidth per competition
competition_intensity_data = {}
competition_best_bandwidth = {}
coords1 = df_fouls[geospatial_features].values
# Define bandwidth search range
grid = GridSearchCV(
    KernelDensity(kernel='gaussian'),
    {'bandwidth': bandwidths},
    cv=5,
    n_jobs=-1
)
grid.fit(coords1)

# Print tested bandwidths and their mean cross-validation scores

for mean_score, std_score, bw in zip(grid.cv_results_['mean_test_score'],
                                     grid.cv_results_['std_test_score'],
                                     grid.cv_results_['param_bandwidth']):
    print(f"  Bandwidth: {bw:.2f}, Score: {mean_score:.4f} ± {std_score:.4f}")


**Quadrat test**

testing to see how the model performs in different areas of the pitch with an 80/20 test train split. N.B. ideally this would be split more carefully but we only need a general sense in this case

This section performs a Quadrat test to evaluate how well the estimated intensity map (derived from Kernel Density Estimation) fits the observed foul locations across different areas of the pitch. The pitch is divided into a grid of quadrats, and the observed number of fouls in each quadrat is compared to the expected number of fouls based on the estimated intensity function.

To provide a more robust evaluation, a train-test split is applied to the foul data. The Kernel Density Estimation model is fitted only on the training data, and the observed foul counts for the Quadrat test are calculated from the independent test data. This approach helps to assess the model's ability to generalize and predict foul locations on unseen data. The chi-squared statistic and its corresponding p-value from the Quadrat test indicate whether there is a statistically significant difference between the observed and expected foul distributions, suggesting a potential lack of fit of the estimated intensity map.

In [ ]:

# === Filter foul events ===
df_fouls = df_concatenated2_2015_2016[df_concatenated2_2015_2016['is_foul_committed'] == 1].copy()

unique_competitions = df_fouls['competition'].unique()

# Pitch setup (assuming you have mplsoccer installed)
pitch = Pitch(pitch_color='grass', line_color='white', stripe=True)

# Define the bandwidths to test
test_bandwidths = [1, 2, 3, 4, 5]

# List to store results for all competitions and bandwidths
quadrat_test_results = []

for competition in unique_competitions:
    print(f"\nProcessing data for: {competition}")
    df_comp_fouls = df_fouls[df_fouls['competition'] == competition].copy()

    # Extract coordinates for the current competition (full dataset)
    coords_full = df_comp_fouls[['x2', 'y2']].values
    N_obs_full = len(coords_full) # Total number of fouls for this competition

    if N_obs_full < 2:
        print("Not enough data for KDE and Quadrat Test. Skipping.")
        continue

    # --- Perform train-test split ---
    # Standard 80% training / 20% test split.
    # 'random_state' ensures reproducibility of the split.

    coords_train, coords_test = train_test_split(coords_full, test_size=0.2, random_state=42)
    N_obs_train = len(coords_train)
    N_obs_test = len(coords_test)

    if N_obs_train < 2 or N_obs_test < 2:
        print(f"Not enough data in train/test sets after split for {competition}. Skipping.")
        continue

    for bandwidth in test_bandwidths:
        print(f"  Testing Bandwidth: {bandwidth}")

        # === KDE fit (ONLY on training data) ===
        kde = KernelDensity(kernel='gaussian', bandwidth=bandwidth)
        kde.fit(coords_train) # <--- KDE fitted on training data only

        # Grid for evaluating the intensity function (full pitch)
        X_grid, Y_grid = np.meshgrid(np.linspace(0, 120, 240), np.linspace(0, 80, 160))
        grid_coords = np.vstack([X_grid.ravel(), Y_grid.ravel()]).T

        log_dens = kde.score_samples(grid_coords)
        pdf = np.exp(log_dens).reshape(X_grid.shape)

        # === Convert density to intensity ===
        # lambda_hat represents the estimated intensity function .
        lambda_hat = N_obs_full * pdf

        # === Quadrat test (using the TEST data for observed counts) ===
        nx, ny = 6, 4 # Number of quadrats (rows, columns)
        x_edges = np.linspace(0, 120, nx+1)
        y_edges = np.linspace(0, 80, ny+1)

        # **Observed counts** are now calculated from the **TEST set**
        obs_counts = np.zeros((ny, nx), dtype=int)
        for i in range(nx):
            for j in range(ny):
                in_tile = (
                    (coords_test[:,0] >= x_edges[i]) & (coords_test[:,0] < x_edges[i+1]) &
                    (coords_test[:,1] >= y_edges[j]) & (coords_test[:,1] < y_edges[j+1])
                )
                obs_counts[j, i] = in_tile.sum()

        # **Expected counts** are derived from the `lambda_hat` (which was fitted on training data
        # but represents the intensity over the full domain).
        exp_counts = np.zeros_like(obs_counts, dtype=float)
        dx_fine_grid = 120 / (X_grid.shape[1] - 1)
        dy_fine_grid = 80 / (Y_grid.shape[0] - 1)
        cell_area_fine_grid = dx_fine_grid * dy_fine_grid

        for i in range(nx):
            for j in range(ny):
                mask = (
                    (X_grid >= x_edges[i]) & (X_grid < x_edges[i+1]) &
                    (Y_grid >= y_edges[j]) & (Y_grid < y_edges[j+1])
                )
                # Summing lambda_hat over the quadrat area gives the expected rate for that area.
                exp_counts[j, i] = np.sum(lambda_hat[mask]) * cell_area_fine_grid

        # IMPORTANT: Scale expected counts to the size of the test set for a direct comparison
        # with 'obs_counts' (which came from the test set). The sum of 'exp_counts' (from lambda_hat)
        # would sum to N_obs_full. We need to scale it down proportionally to N_obs_test.
        scale_factor = N_obs_test / N_obs_full
        exp_counts_scaled = exp_counts * scale_factor

        valid = exp_counts_scaled > 1e-6 # Filter quadrats with extremely small expected counts (now based on scaled exp_counts)

        # Handle cases where no valid quadrats or degrees of freedom are not positive
        if valid.sum() <= 1: # Need at least 2 valid quadrats for df > 0 for chi-squared test
            chi2_stat, df, p_val = np.nan, np.nan, np.nan
            print(f"  Quadrat Test Skipped (BW={bandwidth}): Not enough valid quadrats for calculation or df <= 0.")
        else:
            chi2_stat = np.sum((obs_counts[valid] - exp_counts_scaled[valid])**2 / exp_counts_scaled[valid])
            df = valid.sum() - 1 # Degrees of freedom: num valid quadrats - 1 (for estimated total count from scaled data)
            p_val = chi2.sf(chi2_stat, df)
            print(f"  Quadrat chi2 = {chi2_stat:.2f}, df={df}, p={p_val:.3f}")

        # Store results
        quadrat_test_results.append({
            'competition': competition,
            'bandwidth': bandwidth,
            'chi2_stat': chi2_stat,
            'p_value': p_val
        })

# Convert results to DataFrame and print
df_quadrat_results = pd.DataFrame(quadrat_test_results)
print("\n--- Summary of Quadrat Test Results (Train/Test Split) ---")
print(df_quadrat_results)


**Foul intensity by league**

creating the intensity maps for each league using chosen bandwidth of 3

In [ ]:

# === Filter foul events ===
df_fouls = df_concatenated2_2015_2016[df_concatenated2_2015_2016['is_foul_committed'] == 1].copy()

unique_competitions = df_fouls['competition'].unique()

pitch = Pitch(pitch_color='grass', line_color='white', stripe=True)

# Dictionary to store intensity data for each competition
competition_intensity_data = {}

# Define the bandwidths to test
test_bandwidths = [3]
# List to store results for all competitions and bandwidths
quadrat_test_results = []

# Determine the number of rows and columns for the 2x3 formation
n_rows = 2
n_cols = 3

# Create subplots for each competition
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 6))
axes = axes.flatten() # Flatten the axes array for easy iteration
for i, competition in enumerate(unique_competitions):
#for competition in unique_competitions:
    print(f"\nProcessing data for: {competition}")
    df_comp_fouls = df_fouls[df_fouls['competition'] == competition].copy()
    coords = df_comp_fouls[['x2', 'y2']].values
    N_obs = len(coords)

    if N_obs < 2:
        print("Not enough data for KDE and Quadrat Test. Skipping.")
        continue

    for bandwidth in test_bandwidths:
        print(f"  Testing Bandwidth: {bandwidth}")

        # === KDE fit ===
        kde = KernelDensity(kernel='gaussian', bandwidth=bandwidth) # Bandwidth changes here
        kde.fit(coords)

        # Grid for evaluation (consistent resolution)
        X_grid, Y_grid = np.meshgrid(np.linspace(0, 120, 240), np.linspace(0, 80, 160))
        grid_coords = np.vstack([X_grid.ravel(), Y_grid.ravel()]).T

        log_dens = kde.score_samples(grid_coords)
        pdf = np.exp(log_dens).reshape(X_grid.shape)

        # === Convert density to intensity ===
        lambda_hat = N_obs * pdf
        competition_intensity_data[competition] = lambda_hat
        # === Plot intensity map (optional, uncomment if you want to see each plot) ===
        ax = axes[i]
        pitch.draw(ax=ax)
        pcm = ax.imshow(lambda_hat, origin='lower', extent=(0, 120, 0, 80),
                        cmap='viridis', aspect='auto', alpha=0.8)
        pitch.arrows(0, 0, 0, 0, width=3, headwidth=8, headlength=5,
                     color='#e21017', ax=ax, zorder=2, label="attacking direction")
        fig.colorbar(pcm, ax=ax, label='Estimated Foul Intensity (λ̂)')
        ax.set_title(f'Foul Intensity Map for {competition}')

        ax.legend(bbox_to_anchor=(0.5, -0.1),facecolor='white', handlelength=5, edgecolor='None', fontsize=10, loc='upper center') # Adjusted fontsize




for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

fig.suptitle('Estimated Foul Intensity by League (2015/2016 Season)', y=1.02, fontsize=16) # Add a main title
plt.tight_layout()
plt.show()


**simulating a game**

Use the thinning algorithm to simulates how a distribution of fouls might look in a game

This section utilizes the estimated foul intensity maps and the thinning algorithm to simulate the spatial distribution of fouls in a single "normal" game for each league. The thinning algorithm is a method for generating points from a non-homogeneous Poisson Point Process given an estimated intensity function.

The key idea here is to scale the overall intensity function (derived from all fouls in the season) so that its integral over the pitch area equals the average number of fouls observed per game in that specific league. By applying the thinning algorithm with this scaled intensity, we can generate a simulated set of foul locations that represents a plausible spatial pattern for a typical match in that competition, based on the observed season-long distribution and average foul rate.

In [ ]:
summary = df_fouls.groupby('competition').agg(
    total_rows=('match_id', 'size'),
    unique_match_ids=('match_id', 'nunique')
).reset_index()

summary['average']= summary['total_rows']/summary['unique_match_ids']

In [ ]:

# === Filter foul events ===
df_fouls = df_concatenated2_2015_2016[df_concatenated2_2015_2016['is_foul_committed'] == 1].copy()
unique_competitions = df_fouls['competition'].unique()

# Pitch setup
pitch = Pitch(pitch_color='grass', line_color='white', stripe=True)

# Define the bandwidths to test
test_bandwidths = [3]
# Create a figure with a 2x3 grid of subplots
fig, ax = plt.subplots(2, 3, figsize=(15, 6))
axes = ax.flatten()
# List to store results for all competitions and bandwidths
quadrat_test_results = []
for i, competition in enumerate(unique_competitions):
    current_ax = axes[i]
    #print(f"\nProcessing data for: {competition}")
    df_comp_fouls = df_fouls[df_fouls['competition'] == competition].copy()

    coords_full = df_comp_fouls[['x2', 'y2']].values
    N_obs_full = len(coords_full)

    foul_df=summary[summary['competition'] == competition].copy()
    average_fouls_per_game_assumption = foul_df['average'].iloc[0]

    for bandwidth in test_bandwidths:
        #print(f"  Testing Bandwidth: {bandwidth}")

        # KDE fit
        kde = KernelDensity(kernel='gaussian', bandwidth=bandwidth)

        kde.fit(coords_full)

        # Grid for evaluation (consistent resolution)
        X_grid, Y_grid = np.meshgrid(np.linspace(0, 120, 240), np.linspace(0, 80, 160))
        grid_coords = np.vstack([X_grid.ravel(), Y_grid.ravel()]).T

        log_dens = kde.score_samples(grid_coords)
        pdf = np.exp(log_dens).reshape(X_grid.shape)

        # === Convert density to intensity for the FULL dataset ===
        # This lambda_hat represents the overall spatial intensity for the entire period.
        lambda_hat_full_data = N_obs_full * pdf

        # === NHPPP simulation for a "normal game" ===
        # Step 1: Scale the intensity function so its integral represents the average fouls in ONE game.
        # This is the key change for simulating a "normal game".
        # The scale factor applies to the lambda_hat_full_data so that its integral (sum over pitch area)
        # will equal 'average_fouls_per_game_assumption'.
        lambda_hat_game_scaled = (average_fouls_per_game_assumption / N_obs_full) * lambda_hat_full_data

        # Step 2: Set up the interpolator using the game-scaled intensity
        interp_game = RegularGridInterpolator(
            (np.linspace(0, 120, 240), np.linspace(0, 80, 160)),
            lambda_hat_game_scaled.T, # Use the game-scaled intensity here
            bounds_error=False,
            fill_value=0.0
        )

        # Step 3: Perform thinning using the game-scaled intensity
        A = 120 * 80 # Total area of the pitch
        lambda_max_game_scaled = np.max(lambda_hat_game_scaled) # Max intensity from the game-scaled lambda_hat

        # N_max is now based on the max intensity of the game-scaled lambda_hat.
        # This will result in an average number of simulated points equal to 'average_fouls_per_game_assumption'.
        N_max_game_sim = np.random.poisson(lam=lambda_max_game_scaled * A)

        U_x = np.random.uniform(0, 120, size=N_max_game_sim)
        U_y = np.random.uniform(0, 80, size=N_max_game_sim)
        candidates = np.column_stack([U_x, U_y])

        lam_vals = interp_game(candidates) # Get intensities at candidate points from game-scaled interpolator

        keep = np.random.uniform(0, 1, size=N_max_game_sim) < (lam_vals / lambda_max_game_scaled)
        sim_points = candidates[keep]

        # === Plot the simulated "normal game" ===
        pitch.draw(ax=current_ax)
        current_ax.scatter(sim_points[:,0], sim_points[:,1], s=6, alpha=0.7, color='red')
        # Display the actual number of simulated points
        current_ax.set_title(f'Simulated Foul Pattern for a Normal {competition} Game', fontsize=8)



# Convert results to DataFrame and print
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

fig.suptitle('Simulated Foul Pattern for a Normal Game', y=1.02, fontsize=16) # Add a main title
plt.tight_layout()
plt.show()

**Comparing leagues**

Take intensity data and compare the ratio across the 2d surface

In [ ]:

# Take intensity data
competitions_to_plot = [comp for comp, intensity in competition_intensity_data.items() if intensity is not None]
competitions_with_data = [(comp, data) for comp, data in competition_intensity_data.items() if data is not None]
if len(competitions_with_data) >= 2:
    # First pass: collect all ratio surfaces to find global vmin and vmax
    ratio_surfaces = []
    pairs = []
    epsilon = 1e-15

    for (comp1_name, lambda1), (comp2_name, lambda2) in combinations(competitions_with_data, 2):
        if lambda1.shape == lambda2.shape:
            ratio_surface = lambda1 / (lambda2 + epsilon)
            ratio_surfaces.append(ratio_surface)
            pairs.append((comp1_name, comp2_name))
        else:
            print(f"Intensity maps for {comp1_name} and {comp2_name} have different shapes. Skipping.")

    # Find global min and max for the ratio plots
    global_vmin = min(rs.min() for rs in ratio_surfaces)
    global_vmax = max(rs.max() for rs in ratio_surfaces)

    # limit the range to something reasonable
    global_vmin = max(global_vmin, 0.5)
    global_vmax = min(global_vmax, 1.5)

    # Generate all combinations of two competitions with data
    combinations_to_plot = list(combinations(competitions_with_data, 2))

    # Define the layout of the subplots (5 rows, 2 columns)
    n_rows = 5
    n_cols = 2
    plots_per_figure = n_rows * n_cols

    # Calculate the total number of figures needed
    n_figures = (len(combinations_to_plot) + plots_per_figure - 1) // plots_per_figure

    # Iterate through the combinations in chunks and create figures
    combo_index = 0
    for fig_num in range(n_figures):
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 20),
                                 constrained_layout=False)
        axes = axes.flatten()
        pitch = Pitch(pitch_color='grass', line_color='white', stripe=True)

        for i in range(plots_per_figure):
            if combo_index < len(combinations_to_plot):
                (comp1_name, lambda1), (comp2_name, lambda2) = combinations_to_plot[combo_index]
                ax = axes[i]

                # Ensure both intensity maps have the same shape before plotting
                if lambda1.shape == lambda2.shape:
                    # Calculate the ratio surface
                    epsilon = 1e-15  # Small value to prevent division by zero
                    ratio_surface = lambda1 / (lambda2 + epsilon)

                    # Plot Ratio Surface
                    pitch.draw(ax=ax)
                    pcm = ax.imshow(ratio_surface, origin='lower', extent=(0, 120, 0, 80),
                    cmap='mako', aspect='auto', alpha=0.8,vmin=global_vmin, vmax=global_vmax) # Using viridis for ratio
                    pitch.arrows(0, 0,0, 0, width=3,headwidth=8, headlength=5, color='#e21017', ax=ax, zorder=2, label = "attacking direction")

                    ax.legend(bbox_to_anchor=(0.5, -0.1),facecolor='white', handlelength=5, edgecolor='None', fontsize=10, loc='upper center')
                    fig.colorbar(pcm, ax=ax, label='Intensity Ratio')
                    ax.set_title(f'Ratio Surface: {comp1_name} / {comp2_name}', fontsize=12)

                else:
                    ax.set_title(f'Shape mismatch: {comp1_name} vs {comp2_name}', fontsize=12)
                    ax.text(0.5, 0.5, 'Data shapes mismatch', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)

                combo_index += 1
            else:
                # Hide unused subplots
                axes[i].set_visible(True)

        fig.suptitle(f'Foul Intensity Ratio Surfaces (2015/2016 Season) - Page {fig_num + 1}', y=1.01, fontsize=18)
        plt.tight_layout()
        plt.show()
elif len(competitions_to_plot) < 2:
    print("Not enough competitions with intensity data to calculate ratio surfaces.")


**Creating a Gaussian Mixture Model**

The next step is break down the data into similar areas for testing purposes

This section utilizes a Gaussian Mixture Model (GMM) to identify data-driven zones based on the spatial distribution of fouls. GMM is a probabilistic model that assumes the data points are generated from a mixture of several Gaussian distributions. By fitting a GMM to the foul coordinates, we can cluster areas of the pitch that have similar patterns of foul intensity, creating distinct zones for further analysis.

In [ ]:
df_fouls = df_concatenated2_2015_2016[df_concatenated2_2015_2016['is_foul_committed'] == 1].copy()
geospatial_features = ['x2', 'y2']
kde = KernelDensity(kernel='gaussian', bandwidth=3)  # adjust bandwidth
coords = df_comp_fouls[geospatial_features].values
kde.fit(coords)
X_grid, Y_grid = np.meshgrid(np.linspace(0, 120, 240), np.linspace(0, 80, 160))
grid_coords = np.vstack([X_grid.ravel(), Y_grid.ravel()]).T
log_dens = kde.score_samples(grid_coords)
intensity_array = np.exp(log_dens).reshape(X_grid.shape)
# Step 1: Get coordinates of each cell
rows, cols = np.indices(intensity_array.shape)
coords = np.column_stack((cols.ravel(), rows.ravel()))  # shape: (n_cells, 2)
random=0
# Step 2: Get intensities (weights)
weights = intensity_array.ravel()
weights = weights / weights.sum()  # Normalize to sum to 1

# Step 3: Resample points with repetition based on intensity
N = 10000  # Total number of points to sample
rng = np.random.default_rng(random) # Create a reproducible random number generator
sample_indices = rng.choice(len(weights), size=N, p=weights)

sampled_coords = coords[sample_indices]


**Find optimal breakdown**

In [ ]:

bic = []
for k in range(3, 16):
    gmm = GaussianMixture(n_components=k,random_state=random).fit(sampled_coords)
    bic.append(gmm.bic(sampled_coords))

In [ ]:

# Plot the BIC for each k
plt.figure(figsize=(8, 6))
plt.plot(range(3, 16), bic, marker='o')
plt.xlabel('Number of Components (k)')
plt.ylabel('BIC')
plt.title('BIC vs. Number of Components')
plt.xticks(range(3, 16))
plt.grid(True)
plt.show()

**View map**

In [ ]:

# Step 4: Fit GMM
n_components = 8  # Number of zones to learn
gmm = GaussianMixture(n_components=n_components, random_state=random)
gmm.fit(sampled_coords)

# Step 5: Predict zone labels for full pitch
all_labels = gmm.predict(coords)
zone_map = all_labels.reshape(intensity_array.shape)

# Step 6: Plot the resulting zones
pitch = Pitch(pitch_color='white', line_color='black')
fig, ax = pitch.draw(figsize=(10, 6))
cmap = plt.get_cmap('tab10')

# Visualize the zones
im = ax.imshow(zone_map, origin='lower', extent=(0, 120, 0, 80), cmap=cmap, alpha=0.6)
cbar = plt.colorbar(im, ax=ax, fraction=0.035)
cbar.set_label('GMM Zone')

plt.title('Data-driven Zones from GMM on Foul Intensity Map')
plt.show()

**Chosen zones**

Chosen zones are selected on the basis of the generated zones with additional tactical and interpretable considerations such as the attacking and defensive boxes.

In [ ]:

#Define zones as before, with main and additional rectangles per zone
zones = {
    1: [(0, 0, 36, 36), (36, 0, 80, 80)],
    2: [(0, 124, 36, 160), (36, 80, 80, 160)],
    3: [(80, 0, 160, 53)],
    4: [(80, 53, 160, 107)],
    5: [(80, 107, 160, 160)],
    6: [(204, 0, 240, 36), (160, 0, 204, 80)],
    7: [(204, 124, 240, 160), (160, 80, 204, 160)],
    8: [(0, 36, 36, 124)],
    9: [(204, 36, 240, 124)]
}

fig, ax = plt.subplots(figsize=(12, 7))

# Pitch dimensions for plotting
pitch_length = 240
pitch_width = 160

# Draw pitch outline
ax.plot([0, pitch_length, pitch_length, 0, 0], [0, 0, pitch_width, pitch_width, 0], color='black')

# Colors for zones (repeat if needed)
colors = plt.cm.get_cmap('tab10', 10)

for zone_id, rects in zones.items():
    for idx, (start_col, start_row, end_col, end_row) in enumerate(rects):
        width = end_col - start_col
        height = end_row - start_row

        rect_patch = patches.Rectangle(
            (start_col, start_row),
            width,
            height,
            linewidth=2,
            edgecolor=colors(zone_id),
            facecolor=colors(zone_id),
            alpha=0.3
        )
        ax.add_patch(rect_patch)

        # Label zones in the middle of each rectangle
        ax.text(
            start_col + width / 2,
            start_row + height / 2,
            f"{zone_id}" if idx == 0 else "",  # Label only once per zone
            color='black',
            fontsize=14,
            fontweight='bold',
            ha='center',
            va='center'
        )

ax.set_xlim(0, 240)
ax.set_ylim(0, 160)
ax.set_xlabel("Pitch Length")
ax.set_ylabel("Pitch Width")
ax.set_title("Selected zones for analysis")

plt.gca().set_aspect('equal', adjustable='box')
plt.show()

**Energy distance and permutation tests**

Energy distance measures the difference between two arrays in 2d

Permutations test is an iterative process. The distance is measured. THe 2 sets of datapoints get shuffled up. The new distance is measured. If the random distance is less than the true distribution it is counted as a statistical difference. This counts up for each iteration so the more times the random difference is less the higher the chance of statistical difference.

In [ ]:
def energy_distance_statistic(X, Y):
    """
    Calculate the energy distance between two samples X and Y.
    X and Y should be 1D or 2D numpy arrays.
    """
    X = np.atleast_2d(X)
    Y = np.atleast_2d(Y)

    n, m = len(X), len(Y)

    # Pairwise distances
    XY_dist = cdist(X, Y, metric='euclidean')
    XX_dist = cdist(X, X, metric='euclidean')
    YY_dist = cdist(Y, Y, metric='euclidean')

    term1 = (2.0 / (n * m)) * np.sum(XY_dist)
    term2 = (1.0 / (n * n)) * np.sum(XX_dist)
    term3 = (1.0 / (m * m)) * np.sum(YY_dist)

    energy_dist = term1 - term2 - term3
    return energy_dist

def permutation_test_energy_distance(data1, data2, n_permutations=1000, random_state=None):
    """
    Perform a permutation test using the energy distance statistic.

    Args:
      data1 (array-like): First sample.
      data2 (array-like): Second sample.
      n_permutations (int): Number of permutations.
      random_state (int or None): Seed for reproducibility.

    Returns:
      observed_stat (float): Observed energy distance.
      p_value (float): Permutation p-value.
    """
    if random_state is not None:
        np.random.seed(random_state)

    data1 = np.asarray(data1)
    data2 = np.asarray(data2)

    observed_stat = energy_distance_statistic(data1, data2)

    combined = np.concatenate([data1, data2])
    n1 = len(data1)

    count = 0
    for _ in range(n_permutations):
        permuted = np.random.permutation(combined)
        perm_sample1 = permuted[:n1]
        perm_sample2 = permuted[n1:]
        stat = energy_distance_statistic(perm_sample1, perm_sample2)

        if stat >= observed_stat:
            count += 1

    p_value = count / n_permutations

    return observed_stat, p_value

In [ ]:

zones = {
    1: [(0, 0, 18, 18), (18, 0, 40, 40)],
    2: [(0, 62, 18, 80), (18, 40, 40, 80)],
    3: [(40, 0, 80, 26)],
    4: [(40, 26, 80, 53)],
    5: [(40, 53, 80, 80)],
    6: [(102, 0, 120, 18), (80, 0, 102, 40)],
    7: [(102, 62, 120, 80), (80, 40, 102, 80)],
    8: [(0, 18, 18, 62)],
    9: [(102, 18, 120, 62)]
}
# --- PART 2: Permutation Test (Zonal Comparison) ---
# This dictionary will store raw foul coordinates per league and per zone.
league_zone_coordinates = {}

for competition in unique_competitions:
    league_zone_coordinates[competition] = {}
    df_comp_fouls = df_fouls[df_fouls['competition'] == competition].copy()
    coords = df_comp_fouls[['x2', 'y2']].values # Extract (x,y) coordinates for the league

    for zone_id, rects in zones.items():
        zone_points = []
        for (start_col, start_row, end_col, end_row) in rects:
            # Filter points that fall within the zone's rectangle(s)
            # Note: Assuming pitch (0,0) is bottom-left, x increases right, y increases up.
            in_zone = (
                (coords[:, 0] >= start_col) & (coords[:, 0] < end_col) &
                (coords[:, 1] >= start_row) & (coords[:, 1] < end_row)
            )
            zone_points.append(coords[in_zone])

        # Combine points from all rectangles within the zone
        if zone_points:
            # np.vstack combines arrays vertically if they have compatible shapes
            league_zone_coordinates[competition][zone_id] = np.vstack(zone_points)
        else:
            # If no points found for a zone, store an empty array
            league_zone_coordinates[competition][zone_id] = np.array([])

# This list will store results from all zones, AFTER their per-zone correction
all_corrected_permutation_results = []
leagues = list(unique_competitions) # Get a list of all unique league names

# Loop through each zone to perform pairwise comparisons and then apply Holm's correction
for zone_id in sorted(zones.keys()):
    print(f"\nComparing Zone {zone_id}")

    # Lists to collect p-values and their associated comparison info for the CURRENT ZONE ONLY
    current_zone_p_values = []
    current_zone_comparisons_info = []

    # Perform all pairwise comparisons within the current zone
    for i in range(len(leagues)):
        for j in range(i + 1, len(leagues)): # Ensures each pair is tested only once (e.g., A vs B, not B vs A)
            league1, league2 = leagues[i], leagues[j]

            # Get data for the two leagues in the current zone
            # .get() with a default empty array handles cases where a league might not have data for a specific zone
            data1 = league_zone_coordinates[league1].get(zone_id, np.array([]))
            data2 = league_zone_coordinates[league2].get(zone_id, np.array([]))

            # Skip comparison if either league has too few data points in this zone
            # Energy distance requires at least 2 points for calculation
            if len(data1) < 2 or len(data2) < 2:
                print(f"  Skipping {league1} vs {league2} in Zone {zone_id}: insufficient data (needs >= 2 points).")
                continue

            print(f"  Testing {league1} vs {league2}")
            # Run the permutation test for this specific pair in this zone
            observed_diff, p_value = permutation_test_energy_distance(data1, data2, n_permutations=1000)

            # Collect the raw p-value and all relevant comparison details
            current_zone_p_values.append(p_value)
            current_zone_comparisons_info.append({
                'comparison': f"{league1} vs {league2}",
                'zone': zone_id,
                'observed_diff': observed_diff,
                'p_value': p_value # Store original p-value before correction
            })

    # --- Apply Holm Correction PER ZONE ---
    if current_zone_p_values: # Only proceed if there were valid comparisons in this zone
        p_values_array = np.array(current_zone_p_values)

        # Perform Holm-Bonferroni correction
        reject_zone, pvals_corrected_zone, _, _ = multipletests(p_values_array, method='holm')

        # Attach the corrected p-values and significance flags to the comparison info
        for idx, info in enumerate(current_zone_comparisons_info):
            info['p_value_corrected'] = pvals_corrected_zone[idx]
            info['significant (corrected)'] = reject_zone[idx]
            # Add these results to the overall list of all corrected results
            all_corrected_permutation_results.append(info)
    else:
        print(f"No valid comparisons were made for Zone {zone_id} to apply correction.")

# === Final Results Display for Per-Zone Corrected P-values ===
# Convert the list of dictionaries into a Pandas DataFrame for easy viewing
df_permutation_results_corrected_per_zone = pd.DataFrame(all_corrected_permutation_results)

if not df_permutation_results_corrected_per_zone.empty:
    print("\nPermutation Test Results (Comparing Foul Locations per Zone - Holm-corrected PER ZONE):")
    # Display the full DataFrame with original and corrected p-values
    print(df_permutation_results_corrected_per_zone)

    # Filter for comparisons that are significant after correction
    significant_comparisons_per_zone = df_permutation_results_corrected_per_zone[df_permutation_results_corrected_per_zone['significant (corrected)']]

    print("\nSignificant differences (Holm-corrected p < 0.05 PER ZONE):")
    if not significant_comparisons_per_zone.empty:
        # Display only the significant comparisons
        print(significant_comparisons_per_zone)
    else:
        print("No significant differences found after per-zone correction.")
else:
    print("No pairs of competitions with data to compare across all zones.")



In [ ]:
df_permutation_results_corrected_per_zone['significant']=df_permutation_results_corrected_per_zone['p_value']<0.05
df_permutation_results_corrected_per_zone

**plot results by zone**

In [ ]:

zones = {
    1: [(0, 0, 36, 36), (36, 0, 80, 80)],
    2: [(0, 124, 36, 160), (36, 80, 80, 160)],
    3: [(80, 0, 160, 53)],
    4: [(80, 53, 160, 107)],
    5: [(80, 107, 160, 160)],
    6: [(204, 0, 240, 36), (160, 0, 204, 80)],
    7: [(204, 124, 240, 160), (160, 80, 204, 160)],
    8: [(0, 36, 36, 124)],
    9: [(204, 36, 240, 124)]
}

field_width, field_height = 240, 160

df_permutation_results_corrected_per_zone['zone'] = df_permutation_results_corrected_per_zone['zone'].astype(int)
comparisons = df_permutation_results_corrected_per_zone['comparison'].unique()

n = len(comparisons)
cols = 2  # number of columns for subplot grid
rows = (n + cols - 1) // cols  # ceiling division for rows

fig, axes = plt.subplots(rows, cols, figsize=(cols*6, rows*5))
axes = axes.flatten()  # flatten in case of multiple rows
cmap = plt.get_cmap('Greys')   # Greyscale colormap
norm = plt.Normalize(vmin=0, vmax=1)  # 0 = black, 1 = white
for i, comparison in enumerate(comparisons):
    df_comp = df_permutation_results_corrected_per_zone[df_permutation_results_corrected_per_zone['comparison'] == comparison]
    field = np.ones((field_height, field_width))  # 1 = not significant by default

    for zone_id, rects in zones.items():
        row = df_comp[df_comp['zone'] == zone_id]

        if row.empty:
            is_sig = False
        else:
            is_sig = row['significant'].any()

        value = 0 if is_sig else 1  # 0 = significant (black), 1 = not significant (white)

        for (x0, y0, x1, y1) in rects:
            field[y0:y1, x0:x1] = value
    ax = axes[i]
    im = ax.imshow(field, cmap=cmap, origin='upper', norm=norm)
    ax.set_title(f"Significance Map\n{comparison}", fontsize=10)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlim(0, field_width)
    ax.set_ylim(field_height, 0)

    # Draw zone outlines
    for rects in zones.values():
        for (x0, y0, x1, y1) in rects:
            rect = plt.Rectangle((x0, y0), x1 - x0, y1 - y0,
                                 edgecolor='black', facecolor='none', linewidth=0.5)
            ax.add_patch(rect)

    # Add individual colorbar for each subplot
    sm = ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    fig.colorbar(sm, ax=ax, orientation='vertical', fraction=0.02, pad=0.01,
                 label='0 = significant, 1 = not significant')

# Remove unused axes
for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

In [ ]:

zones = {
    1: [(0, 0, 36, 36), (36, 0, 80, 80)],
    2: [(0, 124, 36, 160), (36, 80, 80, 160)],
    3: [(80, 0, 160, 53)],
    4: [(80, 53, 160, 107)],
    5: [(80, 107, 160, 160)],
    6: [(204, 0, 240, 36), (160, 0, 204, 80)],
    7: [(204, 124, 240, 160), (160, 80, 204, 160)],
    8: [(0, 36, 36, 124)],
    9: [(204, 36, 240, 124)]
}

field_width, field_height = 240, 160

df_permutation_results_corrected_per_zone['zone'] = df_permutation_results_corrected_per_zone['zone'].astype(int)
comparisons = df_permutation_results_corrected_per_zone['comparison'].unique()

n = len(comparisons)
cols = 2
rows = (n + cols - 1) // cols

fig, axes = plt.subplots(rows, cols, figsize=(cols*6, rows*5))
axes = axes.flatten()
cmap = plt.get_cmap('Greys_r')
norm = Normalize(vmin=0, vmax=1.0)  # <--- Use a linear scale from 0 to 1

for i, comparison in enumerate(comparisons):
    df_comp = df_permutation_results_corrected_per_zone[df_permutation_results_corrected_per_zone['comparison'] == comparison]
    field = np.ones((field_height, field_width))

    for zone_id, rects in zones.items():
        row = df_comp[df_comp['zone'] == zone_id]

        if row.empty or row['p_value'].iloc[0] == np.nan:
            value = 1.0
        else:
            value = row['p_value'].iloc[0]

        for (x0, y0, x1, y1) in rects:
            field[y0:y1, x0:x1] = value

    ax = axes[i]
    im = ax.imshow(field, cmap=cmap, origin='upper', norm=norm)
    ax.set_title(f"P-value Map\n{comparison}", fontsize=10)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlim(0, field_width)
    ax.set_ylim(field_height, 0)

    for rects in zones.values():
        for (x0, y0, x1, y1) in rects:
            rect = plt.Rectangle((x0, y0), x1 - x0, y1 - y0,
                                 edgecolor='black', facecolor='none', linewidth=0.5)
            ax.add_patch(rect)

    sm = ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = fig.colorbar(sm, ax=ax, orientation='vertical', fraction=0.02, pad=0.01)
    cbar.set_label('p-value')

for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

**Stouffers z method test**

Calculate the foul weights and combine by compared competition to determine the value is each statistical zone. Then apply Stouffers Z method and adjust by Holm.


In [ ]:
zones_halved = {
    1: [(0, 0, 18, 18), (18, 0, 40, 40)],
    2: [(0, 62, 18, 80), (18, 40, 40, 80)],
    3: [(40, 0, 80, 26)],
    4: [(40, 26, 80, 53)],
    5: [(40, 53, 80, 80)],
    6: [(102, 0, 120, 18), (80, 0, 102, 40)],
    7: [(102, 62, 120, 80), (80, 40, 102, 80)],
    8: [(0, 18, 18, 62)],
    9: [(102, 18, 120, 62)]
}

def assign_zone(row, zones):
    x, y = row['x2'], row['y2']
    for zone_id, rects in zones.items():
        for (x0, y0, x1, y1) in rects:
            # Scale the pitch coordinates (0-120, 0-80) to zone coordinates (0-240, 0-160)
            scaled_x = x
            scaled_y = y

            if x0 <= scaled_x < x1 and y0 <= scaled_y < y1:
                return zone_id
    return None # Return None if the foul is outside all defined zones

df_fouls['zone_id'] = df_fouls.apply(lambda row: assign_zone(row, zones_halved), axis=1)

# Filter out rows where zone_id could not be assigned
df_fouls_zoned = df_fouls.dropna(subset=['zone_id']).copy()

# Count fouls per zone by competition
foul_counts_by_zone_comp = df_fouls_zoned.groupby(['competition', 'zone_id']).size().reset_index(name='foul_count')

print("Number of fouls per zone by competition:")
foul_counts_by_zone_comp

In [ ]:
competition_combinations = [
    ('Spain - La Liga', 'England - Premier League'),
    ('Spain - La Liga', 'France - Ligue 1'),
    ('Spain - La Liga', 'Italy - Serie A'),
    ( 'Germany - 1. Bundesliga','Spain - La Liga'),
    ('France - Ligue 1','England - Premier League'),
    ('England - Premier League', 'Italy - Serie A'),
    ('Germany - 1. Bundesliga','England - Premier League'),
    ('France - Ligue 1', 'Italy - Serie A'),
    ('Germany - 1. Bundesliga','France - Ligue 1'),
    ('Germany - 1. Bundesliga','Italy - Serie A'),
]

combined_foul_counts = []

# Iterate through each desired combination
for comp1, comp2 in competition_combinations:
    # Filter foul counts for the two competitions in the current combination
    df_comp1 = foul_counts_by_zone_comp[foul_counts_by_zone_comp['competition'] == comp1].copy()
    df_comp2 = foul_counts_by_zone_comp[foul_counts_by_zone_comp['competition'] == comp2].copy()

    # Merge the two dataframes on zone_id to align counts for each zone
    # Use outer merge to keep all zones, even if a zone has zero fouls in one competition
    df_combined = pd.merge(df_comp1, df_comp2, on='zone_id', how='outer', suffixes=(f'_{comp1}', f'_{comp2}')).fillna(0)

    # Calculate the sum of foul counts for each zone in this combination
    df_combined['total_foul_count'] = df_combined[f'foul_count_{comp1}'] + df_combined[f'foul_count_{comp2}']

    # Add the comparison name
    df_combined['comparison'] = f'{comp1} vs {comp2}'

    # Select and reorder columns for the result
    result_cols = ['comparison', 'zone_id', 'total_foul_count']
    df_combined_subset = df_combined[result_cols]

    combined_foul_counts.append(df_combined_subset)

# Concatenate all combined results into a single DataFrame
df_combined_foul_counts = pd.concat(combined_foul_counts, ignore_index=True)

print("Combined foul counts by zone_id for unique competition combinations:")
df_combined_foul_counts


In [ ]:

def stouffers_z_method_weighted(p_values, weights):
    """
    Applies weighted Stouffer's Z-score method to combine p-values.

    Args:
        p_values (array-like): List or array of p-values.
        weights (array-like): Corresponding weights (e.g., foul counts) for each p-value.

    Returns:
        tuple: (combined_z, combined_p)
    """
    p_values = np.clip(p_values, 1e-15, 1 - 1e-15)  # Avoid extremes
    z_scores = norm.isf(p_values)  # Convert to Z-scores (one-sided test)

    weights = np.array(weights)
    weighted_z = np.sum(weights * z_scores) / np.sqrt(np.sum(weights ** 2))
    combined_p = norm.sf(weighted_z)  # One-sided p-value

    return weighted_z, combined_p

# Merge the two dataframes on 'comparison' and 'zone'
df_merged_stouffer = pd.merge(
    df_permutation_results_corrected_per_zone[['comparison', 'zone', 'p_value','p_value_corrected']],
    df_combined_foul_counts[['comparison', 'zone_id', 'total_foul_count']],
    left_on=['comparison', 'zone'],
    right_on=['comparison', 'zone_id'],
    how='inner' # Use inner merge to only keep zones present in both dataframes
)

# Group by comparison and apply Stouffer's method across zones for each comparison
stouffer_results = []
for comparison, group in df_merged_stouffer.groupby('comparison'):
    p_values_for_stouffer = group['p_value'].values
    weights = group['total_foul_count'].values
    print(weights)
    if len(p_values_for_stouffer) > 0:
        combined_z, combined_p = stouffers_z_method_weighted(p_values_for_stouffer, weights)

        stouffer_results.append({
            'comparison': comparison,
            'stouffer_combined_z': combined_z,
            'stouffer_combined_p': combined_p,
            'total_fouls_in_comparison': weights.sum()
        })

df_stouffer_combined = pd.DataFrame(stouffer_results)

# Optional: Adjust the combined p-values if you are doing multiple Stouffer's tests
if not df_stouffer_combined.empty:
    combined_p_values = df_stouffer_combined['stouffer_combined_p'].values
    reject, pvals_corrected_stouffer, _, _ = multipletests(combined_p_values, method='holm') # or 'fdr_bh'

    df_stouffer_combined['stouffer_p_corrected'] = pvals_corrected_stouffer
    df_stouffer_combined['stouffer_significant'] = reject

print("\nStouffer's Z Method Results (Combined p-values across zones per comparison):")
df_stouffer_combined